In [1]:
import pandas as pd
import numpy as np
import datetime, random

In [2]:
df = pd.read_csv('/Users/saranmedical-smile/Desktop/patr/StandardizingAKI/inpatient 2014-2018 creatinine.csv') #1441707, 4
df.pat_mrn_id = df.pat_mrn_id.str.strip('MR').astype('int')
df.time = pd.to_datetime(df.time)
print(df.dtypes)
df.columns = ['mrn',
              'enc',
              'time',
              'creat']
df.set_index(['mrn', 'enc'], inplace=True)

pat_mrn_id                 int64
pat_enc_csn_id             int64
time              datetime64[ns]
creatinine               float64
dtype: object


In [404]:
#Quick question to ask: say a patient comes in on a separate encounter within 7 days
#and their creatinine has gone up by 50% since their previous encounter. Does that count as AKI?

#((df.groupby(['pat_mrn_id']).shift(-1) - df).time < datetime.timedelta(days=7)).sum()
#((df.groupby(['pat_mrn_id', 'pat_enc_csn_id']).shift(-1) - df).time < datetime.timedelta(days=7)).sum()

In [3]:
delta_df = df.groupby(['mrn', 'enc']).shift(-1) - df
df['delta_creat'] = delta_df['creat'].shift(1)
df['delta_time'] = delta_df['time'].shift(1)
#df[['delta_time, delta_creat']] = delta_df[['time', 'creatinine']]

In [4]:
np.where(np.logical_and(df.delta_creat > df.creat.shift(1)*0.5, df.delta_time < datetime.timedelta(days=2)))
#676, 7397, ...
#df.iloc[7390:7400]

(array([     95,     256,     354, ..., 1441402, 1441418, 1441653]),)

In [5]:
condition1 = np.logical_and(df.delta_creat > 0.3, df.delta_time < datetime.timedelta(hours=48))
condition2 = np.logical_and(df.delta_creat > df.creat.shift(1)*0.5, df.delta_time < datetime.timedelta(days=7))
df['AKI'] = np.logical_or(condition1, condition2)

#Question: Once you've got AKI you've got AKI for the rest of the encounter, right?
# df.loc['MR1000136']

#Condition 1 examples:
# MR1000136, MR1000468, MR1000507

#Condition 2 examples:
#MR1001546, MR1001806, 

#df[np.logical_xor(condition1, condition2)] gives unique examples of C1 or C2

In [6]:
print('creat > 0.3:', (df.delta_creat > 0.3).sum())
print('time < 48hrs:',(df.delta_time < datetime.timedelta(hours=48)).sum())
print('cond1:', np.logical_and(df.delta_creat > 0.3, df.delta_time < datetime.timedelta(hours=48)).sum())

print('creat_t > 1.5*creat_{t-1}:', (df.delta_creat > df.creat.shift(1)*0.5).sum())
print('time < 48hrs:',(df.delta_time < datetime.timedelta(days=7)).sum())
print('cond2:', np.logical_and(df.delta_creat > df.creat.shift(1)*0.5, df.delta_time < datetime.timedelta(days=7)).sum())

print('cond1 OR cond2:', np.logical_or(condition1, condition2).sum())

creat > 0.3: 63893
time < 48hrs: 1193853
cond1: 60601
creat_t > 1.5*creat_{t-1}: 11899
time < 48hrs: 1234986
cond2: 11825
cond1 OR cond2: 62992


In [7]:
print(df.shape, np.unique(df.index.get_level_values(0)).shape, np.unique(df.index.get_level_values(1)).shape)
df.head()

(1441707, 5) (103924,) (204515,)


time  creat  delta_creat      delta_time  \
mrn     enc                                                                 
1000041 115884935 2015-06-14 20:59:00    1.6          NaN             NaT   
        115884935 2015-06-15 07:54:00    0.9         -0.7 0 days 10:55:00   
        115884935 2015-06-16 07:02:00    0.9          0.0 0 days 23:08:00   
        115884935 2015-06-17 07:11:00    0.9          0.0 1 days 00:09:00   
        117378943 2015-07-18 08:39:00    1.1          NaN             NaT   

                     AKI  
mrn     enc               
1000041 115884935  False  
        115884935  False  
        115884935  False  
        115884935  False  
        117378943  False

In [8]:
def eGFR(creat, age, black, female):
    '''
    Calculates the estimated glomerular filtration rate based on the serum creatinine levels, age, sex, and race (black or not black);
    Based on the formula in the paper A New Equation to Estimate Glomerular Filtration Rate (Levey et. Al, 2009) linked below
    
    https://pubmed.ncbi.nlm.nih.gov/19414839/
    
    '''
    #Term 2 - np.clip(creat/(0.9-0.2*female, a_min=1, a_max=None) is the same as taking min(1, creat/k)
    #Term 3 - np.clip(creat/(0.9-0.2*female, a_min=None, a_max=None) is the same as taking max(1, creat/k)
    #where k is the data-derived constant given in the paper: 0.7 for females and 0.9 for males
    
    return 141*(np.clip(creat/(0.9-0.2*female), a_min=1, a_max=None)**(-0.411+0.082*female))*(np.clip(creat/(0.9-0.2*female), a_min=None, a_max=1)**-1.209)*(0.993**age)*(1+female*0.018)*(1+black*0.159)

#Sample test data
creat = np.random.normal(loc=1, scale=0.2, size=10)
age = np.random.normal(loc=55, scale=10, size=10)
black = np.random.rand(10) > 0.5
female = np.random.rand(10) > 0.5

eGFR(creat, age, black, female) 
#values seem pretty reasonable (80-120)

array([105.72125549,  90.15146172, 102.65193151,  96.61077423,
        98.89993195, 113.45758738, 111.88374885,  77.68721777,
        83.66911541,  95.35673989])

In [9]:
firstencs = df.reset_index().drop_duplicates('mrn') #df.groupby(['mrn']).head(1)
df['first_enc'] = [i in firstencs.index for i in range(df.shape[0])]
df[df.first_enc]

,,time,creat,delta_creat,delta_time,AKI,first_enc
mrn,enc,,,,,,
1000041,115884935,2015-06-14 20:59:00,1.60,NaN,NaT,False,True
1000056,111848406,2015-03-12 21:39:00,0.70,NaN,NaT,False,True
1000057,104466981,2014-09-30 15:38:00,0.70,NaN,NaT,False,True
1000077,169350903,2018-04-11 16:59:00,0.80,NaN,NaT,False,True
1000094,98097982,2014-03-30 07:42:00,3.80,NaN,NaT,False,True
...,...,...,...,...,...,...,...
999893,113721630,2015-06-03 08:36:00,0.60,NaN,NaT,False,True
999926,96076796,2014-02-06 07:45:00,0.80,NaN,NaT,False,True
999938,102049353,2014-08-06 07:33:00,1.00,NaN,NaT,False,True


In [10]:
#groupby and drop_duplicates operate similarly - groupby retains index dd doesn't
df.groupby(['mrn']).head(1)
df.reset_index().drop_duplicates('mrn')

,mrn,enc,time,creat,delta_creat,delta_time,AKI,first_enc
0,1000041,115884935,2015-06-14 20:59:00,1.60,NaN,NaT,False,True
12,1000056,111848406,2015-03-12 21:39:00,0.70,NaN,NaT,False,True
22,1000057,104466981,2014-09-30 15:38:00,0.70,NaN,NaT,False,True
27,1000077,169350903,2018-04-11 16:59:00,0.80,NaN,NaT,False,True
29,1000094,98097982,2014-03-30 07:42:00,3.80,NaN,NaT,False,True
...,...,...,...,...,...,...,...,...
1441689,999893,113721630,2015-06-03 08:36:00,0.60,NaN,NaT,False,True
1441691,999926,96076796,2014-02-06 07:45:00,0.80,NaN,NaT,False,True
1441693,999938,102049353,2014-08-06 07:33:00,1.00,NaN,NaT,False,True
1441701,999960,118591616,2015-08-15 08:39:00,0.80,NaN,NaT,False,True


In [11]:
df['age'] = None
df['sex'] = None
mrn_gb = df.groupby(['mrn'])
tmp = mrn_gb.head(1)
tmp['sex'] = np.random.rand(tmp.shape[0]) > 0.5
tmp['age'] = np.random.normal(loc=50, scale=10, size=tmp.shape[0])
df['age'] = tmp['age'] #So this works! I can match by index the ages & sex
df['sex'] = tmp['sex'] #even though df and tmp are different shapes... awesome! 
df['age'] = mrn_gb['age'].transform(lambda x: x.ffill())
df['sex'] = mrn_gb['sex'].transform(lambda x: x.ffill())
df.head()

/Users/saranmedical-smile/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/saranmedical-smile/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


time  creat  delta_creat      delta_time  \
mrn     enc                                                                 
1000041 115884935 2015-06-14 20:59:00    1.6          NaN             NaT   
        115884935 2015-06-15 07:54:00    0.9         -0.7 0 days 10:55:00   
        115884935 2015-06-16 07:02:00    0.9          0.0 0 days 23:08:00   
        115884935 2015-06-17 07:11:00    0.9          0.0 1 days 00:09:00   
        117378943 2015-07-18 08:39:00    1.1          NaN             NaT   

                     AKI  first_enc        age   sex  
mrn     enc                                           
1000041 115884935  False       True  40.823068  True  
        115884935  False      False  40.823068  True  
        115884935  False      False  40.823068  True  
        115884935  False      False  40.823068  True  
        117378943  False      False  40.823068  True

In [364]:
#df[df.delta_creat.isnull()].head(30)
bc = df[np.logical_and(df.delta_creat.isnull(), ~df.first_enc)] # these need to be back-calculated
bc.head()

,mrn,enc,time,creat,delta_creat,delta_time,AKI,first_enc
406306,192,134948062,2016-07-14 20:34:00,1.50,NaN,NaT,False,False
495086,212,145897098,2017-02-10 19:00:00,0.32,NaN,NaT,False,False
495093,212,146461432,2017-02-22 04:32:00,0.34,NaN,NaT,False,False
495101,212,147073095,2017-03-03 18:21:00,0.50,NaN,NaT,False,False
704475,262,136635955,2016-10-31 15:36:00,1.30,NaN,NaT,False,False


In [927]:
#For pat mrn 212
##0 --> 0
#11 --> 0.29818181818181827
#18 --> 0.31500000000000006
#26 --> 0.33666666666666667

pat_df = patient_dfs.get_group(212)
#backcalc_rows = np.where(np.logical_and(pat_df.delta_creat.isnull(), ~pat_df.first_enc))[0]
backcalc_rows = np.where(pat_df.delta_creat.isnull())[0]
print(backcalc_rows)

indx=None
rows_to_add = list()
for indx in backcalc_rows:
    lookback_indices = list()
    init_indx = indx
    
    if indx is not None:
        while (pat_df.iloc[indx].time - pat_df.iloc[indx-1].time) < datetime.timedelta(days=365) and pat_df.iloc[indx].mrn == pat_df.iloc[indx-1].mrn:
            if pat_df.iloc[init_indx].time - pat_df.iloc[indx-1].time > datetime.timedelta(days=7):
                lookback_indices.append(indx-1)
            if indx == 0:
                break
            indx -= 1
    lookback_indices = np.sort(lookback_indices)
    new_row = pat_df.iloc[[init_indx]]
    new_row.loc[:, 'creat'] = pat_df.iloc[lookback_indices].creat.mean()
    rows_to_add.append(new_row)
    
mini_dfs = np.split(pat_df, backcalc_rows)

for indx, mini_df in enumerate(mini_dfs[1:]):
    mini_dfs[indx+1] = pd.concat([rows_to_add[indx], mini_df])
pat_df = pd.concat(mini_dfs)

[ 0 11 18 26]


/Users/saranmedical-smile/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [993]:
patient_dfs = df.groupby(['mrn'])
patients = dict()
for mrn, pat_df in patient_dfs:
    patients[mrn] = pat_df
    backcalc_rows = np.where(pat_df.delta_creat.isnull())[0]

    indx=None
    rows_to_add = list()
    for indx in backcalc_rows:
        lookback_indices = list()
        init_indx = indx

        if indx is not None:
            while (pat_df.iloc[indx].time - pat_df.iloc[indx-1].time) < datetime.timedelta(days=365) and pat_df.iloc[indx].mrn == pat_df.iloc[indx-1].mrn:
                if pat_df.iloc[init_indx].time - pat_df.iloc[indx-1].time > datetime.timedelta(days=7):
                    lookback_indices.append(indx-1)
                if indx == 0:
                    break
                indx -= 1
        lookback_indices = np.sort(lookback_indices)
        new_row = pat_df.iloc[[init_indx]]
        new_row.loc[:, 'creat'] = pat_df.iloc[lookback_indices].creat.mean()
        rows_to_add.append(new_row)

    mini_dfs = np.split(pat_df, backcalc_rows)
    for indx, mini_df in enumerate(mini_dfs[1:]):
        mini_dfs[indx+1] = pd.concat([rows_to_add[indx], mini_df])
    pat_df = pd.concat(mini_dfs)
    patients[mrn] = pat_df


/Users/saranmedical-smile/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [1024]:
pd.concat(list(patients.values()))

,mrn,enc,time,creat,delta_creat,delta_time,AKI,first_enc
238225,154,126559679,2016-02-29 13:03:00,NaN,NaN,NaT,False,True
238225,154,126559679,2016-02-29 13:03:00,0.9,NaN,NaT,False,True
238226,154,126559679,2016-03-01 06:23:00,0.7,-0.2,0 days 17:20:00,False,False
238227,154,126559679,2016-03-02 06:00:00,0.8,0.1,0 days 23:37:00,False,False
246603,156,130878120,2016-05-01 18:03:00,NaN,NaN,NaT,False,True
...,...,...,...,...,...,...,...,...
542372,2225,98666634,2014-05-11 04:55:00,0.4,-0.2,0 days 23:16:00,False,False
542373,2225,98666634,2014-05-12 04:38:00,0.4,0.0,0 days 23:43:00,False,False
542374,2225,98666634,2014-05-13 04:56:00,0.4,0.0,1 days 00:18:00,False,False
542375,2225,98666634,2014-05-14 04:54:00,0.4,0.0,0 days 23:58:00,False,False


In [569]:
for mrn, dataframe in tmp.groupby(['mrn']):
    print(mrn, np.where(np.logical_and(dataframe.delta_creat.isnull(), ~dataframe.first_enc))[0])

154 []
156 []
192 [3]
211 []
212 [11 18 26]
262 [4]
266 [4]
299 []
321 []
418 []
440 [ 5 10 14]
541 []
550 [10]
564 []
592 [10]
609 []
611 [2]
618 [2 6]
630 []
645 []
646 [3 5 8]
666 []
680 []
696 []
699 []
700 []
701 []
738 [2]
751 []
770 [ 7 17 29 40]
783 [ 4  7 12 14]
798 [ 50  63 200 209 213 222 228 235 254]
810 [24]
886 [2 6]
889 [8]
910 []
912 [ 4 12 19 29 32 40 43 58 61 70 73 75 78]
925 [19 22 28 33 38 43]
956 [ 6 14 17 21]
962 [ 3  7 10 18 20 23]
989 [ 2  4 10 13 15]
1016 [3]
1051 [4]
1078 []
1107 [6]
1134 []
1187 []
1229 []
1249 []
1308 []
1375 []
1391 [2 4]
1442 []
1459 []
1476 []
1539 []
1542 []
1591 []
1628 []
1633 []
1815 []
1824 []
1833 []
1975 [3]
1987 [4]
2019 []
2075 [6]
2184 []
2192 []
2199 [4]
2225 [ 7 13]


{154: Int64Index([238225, 238226, 238227], dtype='int64'),
 156: Int64Index([246603, 246604], dtype='int64'),
 192: Int64Index([406303, 406304, 406305, 406306, 406307, 406308, 406309], dtype='int64'),
 211: Int64Index([491546, 491547, 491548], dtype='int64'),
 212: Int64Index([495075, 495076, 495077, 495078, 495079, 495080, 495081, 495082,
             495083, 495084, 495085, 495086, 495087, 495088, 495089, 495090,
             495091, 495092, 495093, 495094, 495095, 495096, 495097, 495098,
             495099, 495100, 495101, 495102, 495103, 495104, 495105, 495106,
             495107, 495108, 495109, 495110, 495111, 495112, 495113, 495114,
             495115],
            dtype='int64'),
 262: Int64Index([704471, 704472, 704473, 704474, 704475, 704476, 704477, 704478,
             704479, 704480, 704481, 704482, 704483],
            dtype='int64'),
 266: Int64Index([707477, 707478, 707479, 707480, 707481, 707482, 707483, 707484], dtype='int64'),
 299: Int64Index([734857, 734858, 734

In [469]:
#print(lookback_indices, np.sort(lookback_indices))
df.iloc[lookback_indices]

,mrn,enc,time,creat,delta_creat,delta_time,AKI,first_enc
406303,192,132541883,2016-06-01 18:14:00,1.1,NaN,NaT,False,True
406304,192,132541883,2016-06-02 05:32:00,1.1,0.0,0 days 11:18:00,False,False
406305,192,132541883,2016-06-03 05:46:00,1.3,0.2,1 days 00:14:00,False,False


In [483]:
new_row = df.iloc[init_indx].copy()
new_row.creat = df.iloc[lookback_indices].creat.mean()
new_row

mrn                            192
enc                      134948062
time           2016-07-14 20:34:00
creat                      1.16667
delta_creat                    NaN
delta_time                     NaT
AKI                          False
first_enc                    False
Name: 406306, dtype: object

In [565]:
pd.concat([df.iloc[:init_indx], 
           pd.DataFrame(new_row).T, 
           df.iloc[init_indx:]])

,mrn,enc,time,creat,delta_creat,delta_time,AKI,first_enc
238225,154,126559679,2016-02-29 13:03:00,0.9,NaN,NaT,False,True
238226,154,126559679,2016-03-01 06:23:00,0.7,-0.2,0 days 17:20:00,False,False
238227,154,126559679,2016-03-02 06:00:00,0.8,0.1,0 days 23:37:00,False,False
406306,192,134948062,2016-07-14 20:34:00,1.16667,NaN,NaT,False,False
246603,156,130878120,2016-05-01 18:03:00,0.6,NaN,NaT,False,True
...,...,...,...,...,...,...,...,...
542372,2225,98666634,2014-05-11 04:55:00,0.4,-0.2,0 days 23:16:00,False,False
542373,2225,98666634,2014-05-12 04:38:00,0.4,0,0 days 23:43:00,False,False
542374,2225,98666634,2014-05-13 04:56:00,0.4,0,1 days 00:18:00,False,False
542375,2225,98666634,2014-05-14 04:54:00,0.4,0,0 days 23:58:00,False,False
